In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!kaggle datasets download -d mostafaabla/garbage-classification

 91% 218M/239M [00:01<00:00, 155MB/s]
100% 239M/239M [00:01<00:00, 145MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/garbage-classification.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Using Generators
gen_train = ImageDataGenerator(rescale = 1/255, shear_range = 0.3, zoom_range = 0.3,
                              horizontal_flip=True,validation_split = 0.2)

train_data = gen_train.flow_from_directory("/content/garbage_classification",
                                           target_size = (256, 256), batch_size = 32, class_mode="categorical",subset = "training")



val_data = gen_train.flow_from_directory("/content/garbage_classification",
                                           target_size = (256, 256), batch_size = 32, class_mode="categorical",subset = "validation")



Found 12415 images belonging to 12 classes.
Found 3100 images belonging to 12 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
conv_base = VGG16(weights = 'imagenet',include_top = False,input_shape = (256,256,3))

In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(52,activation='relu'))
model.add(Dense(12,activation='softmax'))

In [ ]:
conv_base.trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 52)                1703988   
                                                                 
 dense_1 (Dense)             (None, 12)                636       
                                                                 
Total params: 16419312 (62.63 MB)
Trainable params: 1704624 (6.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics =["accuracy"])

In [ ]:
res = model.fit(train_data,epochs = 10,steps_per_epoch=len(train_data),validation_data=val_data)

Epoch 1/10
 10/388 [..............................] - ETA: 2:38:24 - loss: 2.7585 - accuracy: 0.3562

In [ ]:
from keras.models import load_model
model.save("network.h5")

In [ ]:
loaded_model = load_model("network.h5")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing import image
output_class = ["battery", "biological", "brown-glass", "cardboard", "clothes", "green-glass", "metal", "paper", "plastic","shoes","trash","white-glass"]
def waste_prediction(new_image):
  t_img = image.load_img(new_image, target_size = (256,256))
  plt.axis("off")
  plt.imshow(t_img)
  plt.show()

  t_img = image.img_to_array(t_img)/255
  t_img = np.expand_dims(t_img, axis=0)

  predicted_array = model.predict(t_img)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print("Your waste material is ", predicted_value, " with ", predicted_accuracy, " % accuracy")

In [ ]:
waste_prediction("/content/battery.jpg")